In [ ]:
class VisionEncoder(nn.Module):
    def __init__(self, d_out: int) -> None:
        super().__init__()
        self.base = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        d_in = self.base.config.hidden_size
        self.projection = Projection(d_in, d_out)
        
        for p in self.base.parameters():
            p.requires_grad = False

    def forward(self, x):
        outputs = self.base(pixel_values=x)
        cls_token = outputs.last_hidden_state[:, 0, :]
        projected_vec = self.projection(cls_token)
        projection_len = torch.norm(projected_vec, dim=-1, keepdim=True)
        return projected_vec / projection_len

In [ ]:
from torchvision.models import efficientnet_b0

class VisionEncoder(nn.Module):
    def __init__(self, d_out: int) -> None:
        super().__init__()
        self.base = efficientnet_b0(pretrained=True)
        d_in = self.base.classifier[1].in_features
        self.base.classifier = nn.Identity()

        self.projection = Projection(d_in, d_out)

        for p in self.base.parameters():
            p.requires_grad = False

    def forward(self, x):
        features = self.base(x)
        projected_vec = self.projection(features)
        projection_len = torch.norm(projected_vec, dim=-1, keepdim=True)
        return projected_vec / projection_len

In [ ]:
# FEHLERHAFT
# Tokenizer müssen dafür überall angepasst werden

from transformers import GPT2Model, GPT2Tokenizer

class TextEncoder(nn.Module):
    def __init__(self, d_out: int) -> None:
        super().__init__()
        self.text_model = GPT2Model.from_pretrained("gpt2")
        d_in = self.text_model.config.hidden_size

        self.projection = Projection(d_in, d_out)

        for p in self.text_model.parameters():
            p.requires_grad = False

    def forward(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()} 

        outputs = self.text_model(**inputs)
        last_hidden_states = outputs.last_hidden_state

        last_token = last_hidden_states[:, -1, :]

        projected_vec = self.projection(last_token)

        projection_len = torch.norm(projected_vec, dim=-1, keepdim=True)
        return projected_vec / projection_len